# Street View House Numbers - Evaluation on Test dataset

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import svhn
import graphics
import keras_utils
from keras.utils import np_utils
import keras
from keras import backend as K

%matplotlib inline

max_digits = 7
image_size = (48,116)

# print the keras version used
import keras
print "Keras version : {}".format(keras.__version__)

Using TensorFlow backend.


Keras version : 1.2.0


## Load test dataset
Here we load the sample points as individual prediction targets. We will not flatten the data like we did during training, since we have to predict the complete sequence of digits in an image together.

In [2]:
# load the data file (takes time)
rawdata = svhn.read_process_h5('../inputs/test/digitStruct.mat')

In [3]:
# extract resized images, counts and label sequences for each sample
def generateTestData(data, n=100):
    Ximg = []
    ycount = []
    ylabel = []
    for datapoint in np.random.choice(data, size=n, replace=False):
        img,rawsize = svhn.createImageData(datapoint, image_size, '../inputs/test/')
        Ximg.append(img)
        ycount.append(datapoint['length'])
        ylabel.append(datapoint['labels'])
        
    ylabel = [[0 if y==10 else int(y) for y in ys] for ys in ylabel]
    return np.array(Ximg), np.array(ycount), np.array(ylabel)
    
# change to 13068 to test on full test dataset
Ximg, ycount, ylabel = generateTestData(rawdata, 13068)

## Load models
Our training graph is composed of macro models - vision, counter and label detector. While it was suitable for training on flattened data, we will need to organize the models differently to generate sequence predictions.
Extract the models from parent graph.

In [13]:
model_yaml = open('../checkpoints/model.yaml','r')
model = keras.models.model_from_yaml(model_yaml.read())
model_yaml.close()
model.load_weights('../checkpoints/model.hdf5')

# enumerate the layers of main graph
for i,layer in zip(range(len(model.layers)), model.layers):
    print "layer {} : {}".format(i,layer.name)

layer 0 : train_input_img
layer 1 : vision
layer 2 : train_input_idx
layer 3 : counter
layer 4 : detector


In [14]:
# extract the individual models from training graph
vision = model.layers[1]
counter = model.layers[3]
detector = model.layers[4]

## Prediction
For prediction, we will first generate the intermediate output h, from vision model. We will then pass it to counter first. Then the detector will be called for each sample with all indices in one go. 

The crucial part here is that we want to calculate the intermediate output h only once to save on computations.

In [15]:
h = vision.predict(Ximg)

In [16]:
ycount_ = counter.predict(h)
ycount_ = np.argmax(ycount_, axis=1)

In [17]:
ylabel_ = []
for i in range(len(ycount_)):
    # generate range for each count
    indices = np.arange(ycount_[i])
    # one hot encoding for each index
    indices = np_utils.to_categorical(indices, max_digits)
    # tile h to match shape of indices matrix
    hs = np.tile(h[i], (ycount_[i],1))
    
    # predict labels for the sample
    sample_seq = detector.predict([hs, indices])
    sample_seq = np.argmax(sample_seq,1)
    ylabel_.append(sample_seq)

## Evaluation
We can evaluate the performance of both counter and label detector to get a better insight on the fit. In the end, we will evaluate the performance of the whole system. We will consider a predicted sequence to be correct only if all labels have been identified successfully, as any wrong classification can result in a totally different house number.

### Counter Metrics

In [18]:
from sklearn.metrics import classification_report
print classification_report(ycount, ycount_)

             precision    recall  f1-score   support

          1       0.86      0.47      0.61      2483
          2       0.81      0.80      0.80      8356
          3       0.45      0.71      0.55      2081
          4       0.26      0.43      0.32       146
          5       0.00      0.00      0.00         2

avg / total       0.76      0.72      0.72     13068



### Detector Metrics
Here we need to presume that counter has made a perfect prediction. If the counter itself has predicted wrong count, then it shouldn't be counted as detector's failure. So we'll evaluate the detector's performance for digits in a sequence till an index such that index is the minimum of true count value and predicted count value.

In [19]:
ycmin = np.minimum(ycount, ycount_)

# extract labels from ylabel and ylabel_ using ycmin
ylabel_det = np.array([ylabelrow[0:ycminc] for ylabelrow,ycminc in zip(ylabel, ycmin)])
ylabel_det = np.concatenate(ylabel_det)

ylabel_det_= np.array([ylabelrow[0:ycminc] for ylabelrow,ycminc in zip(ylabel_, ycmin)])
ylabel_det_= np.concatenate(ylabel_det_)

In [20]:
print classification_report(ylabel_det, ylabel_det_)

             precision    recall  f1-score   support

          0       0.55      0.50      0.52      1631
          1       0.54      0.72      0.62      4988
          2       0.58      0.58      0.58      4060
          3       0.48      0.41      0.44      2809
          4       0.50      0.51      0.50      2452
          5       0.50      0.50      0.50      2285
          6       0.50      0.37      0.42      1911
          7       0.55      0.51      0.53      1933
          8       0.48      0.30      0.37      1597
          9       0.40      0.45      0.43      1533

avg / total       0.52      0.52      0.52     25199



### Overall Sequence Prediction Accuracy

In [21]:
def matchSequence(seq, seq_):
    return [np.array_equal(seqi, seqi_) for seqi, seqi_ in zip(seq, seq_)]
seqmatch = matchSequence(ylabel, ylabel_)
print "Sequence prediction accuracy : {}".format(np.average(seqmatch))

Sequence prediction accuracy : 0.326522803796


## Display samples
Show some prediction results. The value in brackets is the true value.

In [ ]:
graphics.displaySamples(Ximg, ycounttrue=ycount, ycountpred=ycount_, ylabels=ylabel, ylabelspred=ylabel_)

## Visualize Convolutions
We can visualize the convolutions in vision model to see how the image is being processed. Change **convlayer** to see the convolutions generated by the desired layer.

In [ ]:
convlayer = 1
graphics.showCNNConv(vision, convlayer, Ximg[19])